In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import config
import logging.config
import logging
import re
import time
import random
import json

class Translator(object):
    def __init__(self):
        self.option = webdriver.ChromeOptions()
        self.option.add_argument('--no-sandbox')
        self.option.add_argument('--disable-dev-shm-usage')
        self.option.add_argument('--start-maximized')
        self.option.add_argument('--disable-infobars')
        self.option.add_argument('--disable-extensions')
        self.option.add_argument('--disable-gpu')
        self.option.add_argument('--ignore-certificate-errors')
        self.option.add_argument('--lang=en')
        self.option.add_experimental_option("prefs", { 
            "profile.default_content_setting_values.notifications": 2 
            })
        self.option.add_argument('--headless')
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=self.option)

    def translate_text(self, src, dest, text):
        driver = self.driver
        url = f'https://translate.google.com/?sl={src}&tl={dest}&op=translate'
        driver.get(url)
        text_area = driver.find_element(By.XPATH, '//textarea[@aria-label="Source text"]')
        text_area.send_keys(text)
        elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div/span[@lang]/span/span[text()]')))
        complete_result = [element.text for element in elements]
        complete_result = ' '.join(complete_result)
        # translated_text = driver.find_element(By.XPATH, '//div/span[@lang]/span/span').text
        return complete_result
    

import json
import os
import config
import mysql.connector

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            # database = config.configSQLCrawler.DATABASE,
            database = 'data_skripsi',
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

trip_to_gtranslate_code = {
    'in' :'id',
    'en' : 'en',
    'de' : 'de',
    'nl' : 'nl',
    'fr' : 'fr',
    'es' : 'es',
    'ru' : 'ru',
    'cs' : 'cs',
    'pt' : 'pt',
    'it' : 'it',
    'da' : 'da',
    'zhTW' : 'zh-TW',
    'ko' : 'ko',
    'ja' : 'ja',
    'ar' : 'ar',
    'zhCN' : 'zh-CN',
    'sv' : 'sv',
    'no' : 'no',
    'pl' : 'pl',
    'el' : 'el',
    'sr' : 'sr',
    'tr' : 'tr',
    'th' : 'th',
    'vi' : 'vi'
}

db = init_db()
cursor = get_cursor(db)

cursor.execute('SELECT id, lang, title, text FROM hotel_review_filtered WHERE lang="in" AND translated_text is null')
all_data = cursor.fetchall()
baris = len(all_data)

translator = Translator()
num_of_char = 0

with open('error_id.txt') as file:
    lines = [line.rstrip() for line in file]

with open('log.txt','a+', newline='') as log_txt:
    processed = 0
    for i, row in enumerate(all_data):
        if ('\n' not in row[3] and str(row[0]) not in lines) and (len(row[3]) < 500 or '"' not in row[3]):
            try:
                translated_text = translator.translate_text('id', 'en', row[3])
                log_txt.write(f'Original: {row[3]}\nTranslated: {translated_text}\n')
                cursor.execute(f'UPDATE hotel_review_filtered SET translated_text="{translated_text}" WHERE id={row[0]}')
                db.commit()
                processed += 1
            except Exception as e:
                with open('error_id.txt', 'a+', newline='') as error_txt:
                    error_txt.write(str(row[0]) + '\n')
                try:
                    print(translated_text)
                except:
                    pass
        print('\r',i+1,'/',baris, 'Processed: ', processed, end='')


In [ ]:
import os

def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target, format_='text')

    print(u"Text: {}".format(result["input"]))
    print(repr("Translation: {}".format(result["translatedText"])))
    print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './skripsi-hasea-9494d44e84dd.json'


In [ ]:
print(lang_list)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import config
import logging.config
import logging
import re
import time
import random
import json

class Translator(object):
    def __init__(self):
        self.option = webdriver.ChromeOptions()
        self.option.add_argument('--no-sandbox')
        self.option.add_argument('--disable-dev-shm-usage')
        self.option.add_argument('--start-maximized')
        self.option.add_argument('--disable-infobars')
        self.option.add_argument('--disable-extensions')
        self.option.add_argument('--disable-gpu')
        self.option.add_argument('--ignore-certificate-errors')
        self.option.add_argument('--lang=en')
        self.option.add_experimental_option("prefs", { 
            "profile.default_content_setting_values.notifications": 2 
            })
        self.option.add_argument('--headless')
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=self.option)

    def translate_text(self, src, dest, text):
        driver = self.driver
        url = f'https://translate.google.com/?sl={src}&tl={dest}&op=translate'
        driver.get(url)
        text_area = driver.find_element(By.XPATH, '//textarea[@aria-label="Source text"]')
        text_area.send_keys(text)
        elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div/span[@lang]/span/span[text()]')))
        complete_result = [element.text for element in elements]
        complete_result = ' '.join(complete_result)
        # translated_text = driver.find_element(By.XPATH, '//div/span[@lang]/span/span').text
        return complete_result

In [ ]:
import pymongo
from pymongo import MongoClient
import json
import os
import config
import mysql.connector

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            # database = config.configSQLCrawler.DATABASE,
            database = 'data_skripsi',
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

trip_to_gtranslate_code = {
    'in' :'id',
    'en' : 'en',
    'de' : 'de',
    'nl' : 'nl',
    'fr' : 'fr',
    'es' : 'es',
    'ru' : 'ru',
    'cs' : 'cs',
    'pt' : 'pt',
    'it' : 'it',
    'da' : 'da',
    'zhTW' : 'zh-TW',
    'ko' : 'ko',
    'ja' : 'ja',
    'ar' : 'ar',
    'zhCN' : 'zh-CN',
    'sv' : 'sv',
    'no' : 'no',
    'pl' : 'pl',
    'el' : 'el',
    'sr' : 'sr',
    'tr' : 'tr',
    'th' : 'th',
    'vi' : 'vi'
}

db = init_db()
cursor = get_cursor(db)

cursor.execute('SELECT id, lang, title, text FROM hotel_review_filtered WHERE lang="in" AND translated_text is null')
all_data = cursor.fetchall()
baris = len(all_data)

translator = Translator()
num_of_char = 0

with open('log.txt','a+', newline='') as log_txt:
    processed = 0
    for i, row in enumerate(all_data):
        if '\n' not in row[3]:
            try:
                translated_text = translator.translate_text('id', 'en', row[3])
                log_txt.write(f'Original: {row[3]}\nTranslated: {translated_text}\n')
                cursor.execute(f'UPDATE hotel_review_filtered SET translated_text="{translated_text}" WHERE id={row[0]}')
                db.commit()
                processed += 1
            except Exception as e:
                print(e)
                print(translated_text)
        print('\r',i+1,'/',baris, 'Processed: ', processed, end='')


In [ ]:
cursor.close()
db.close()

# Cek jumlah karakter


In [1]:
import pymongo
from pymongo import MongoClient
import json
import os
import config
import mysql.connector

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            # database = config.configSQLCrawler.DATABASE,
            database = 'data_skripsi',
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)
db = init_db()
cursor = get_cursor(db)

cursor.execute(f'SELECT id, text FROM hotel_review_filtered WHERE (translated_text is null OR translated_text="") AND NOT lang="en"')
all_data = cursor.fetchall()

jumlah_karakter = 0
for row in all_data:
    jumlah_karakter += len(row[1])

print(jumlah_karakter)

10708258


In [15]:
import pymongo
from pymongo import MongoClient
import json
import os
import config
import mysql.connector

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            # database = config.configSQLCrawler.DATABASE,
            database = 'data_skripsi',
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)
db = init_db()
cursor = get_cursor(db)

cursor.execute(f'SELECT id, text, translated_text FROM hotel_review_filtered WHERE (translated_text is not null OR not translated_text="") AND NOT lang="en"')
all_data = cursor.fetchall()

faulty_translation = list()
for row in all_data:
    baris_original = len(row[1])
    baris_translated = len(row[2])
    ratio = abs(baris_original-baris_translated)/baris_original
    if ratio > 0.2:
        faulty_translation.append(row + (ratio,))

sorted_faulty = sorted(faulty_translation, key=lambda d: d[3], reverse=True)

for faulty in sorted_faulty:
    print(f'Original:\n{faulty[1]}')
    print(f'Translated:\n{faulty[2]}')
    print(f'Ratio:\n{faulty[3]}')
    print('='*70)


Original:
MUANTAP PELAYANANNYA, Big thanks to Om Firman dari Ai, kolam renangnya luas dan bersih, kamar juga nyaman banget, bersih jugaaa, makanannya juga ueenaakkkk puolllll, keren pokoknya, lain kali keseni lagi dehhh whahahahahha
Translated:
Muantap the service, big thanks to om the word from ai, the swimming pool is spacious and clean, the room is also very comfortable, clean too, the food is also ueenaakkkk Puolllll, cool anyway, next time the art is again dehhh whahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahahaceoreeeingiveeingeeingeeepers with on on on ones
Ratio:
1.7207207207207207
Original:
Servicenya terbaikkk, pokoknya servicenya keren deh ramah pelayanannya tempatnya nyaman bersih pokoknya tidak rugi